In [1]:
import gurobipy as gp
import pandas as pd

# Data

In [2]:
cust_sat_matrix = [[90, 82, 0, 55, 44, 78, 86, 57, 83, 80],
                   [84, 92, 99, 78, 81, 90, 99, 85, 96, 80],
                   [99, 95, 91, 27, 33, 84, 87, 56, 97, 48],
                   [96, 93, 90, 39, 42, 92, 89, 61, 99, 54],
                   [99, 95, 91, 27, 33, 84, 87, 56, 97, 48],
                   [86, 94, 99, 76, 83, 90, 97, 85, 98, 78],
                   [99, 95, 91, 27, 33, 84, 87, 56, 97, 48],
                   [99, 95, 91, 27, 33, 84, 87, 56, 97, 48],
                   [90, 82, 0, 55, 44, 78, 86, 57, 83, 80],
                   [90, 95, 80, 40, 40, 90, 90, 70, 99, 60],
                   [96, 93, 90, 39, 42, 92, 89, 61, 99, 54],
                   [99, 95, 91, 27, 33, 84, 87, 56, 97, 48],
                   [86, 94, 99, 76, 83, 90, 97, 85, 98, 78],
                   [99, 95, 91, 27, 33, 84, 87, 56, 97, 48],
                   [99, 95, 91, 27, 33, 84, 87, 56, 97, 48]
]

# Each element in cust_sat_matrix should be 100 minus that element.
cust_sat_matrix_new = [[100 - value for value in row] for row in cust_sat_matrix]
fee_list = [1500, 1400, 1300, 1100, 1100, 0, 0, 1300, 1600, 1200]
num_customer_list = [30, 26, 36, 30, 36, 30, 44, 40, 26, 34, 30, 30, 40, 44, 36]
start_day_list = [1,1,2,4,6,7,8,10,14,16,16,16,17,19,25]
end_day_list = [12,12,13,14,17,17,19,21,25,29,25,28,26,30,37]
tours = range(15)
leaders = range(10)
n_tours = len(tours)
n_leaders = len(leaders)

# Optimization

In [ ]:
def optimize_tours(M, a=2):
    """
    M (int): big M used in objective function to make the satisfaction score have more weight
    a (int): number of vacation days that a tour leader must have between tours
    """

    model = gp.Model("EU Holidays")
    model.Params.outputFlag = 0 # 0 for NOT showing output

    ############## Add Variables ##############
    # x equals 1 if tour t is assigned to tour leader l. 0 otherwise.
    x = model.addVars(tours, leaders, vtype = gp.GRB.BINARY, name="x")

    ############## Add Constraints ##############
    # 1. Each tour is only assigned to a single leader. This also ensures that every tour is assigned.
    for t in tours:
        model.addConstr(gp.quicksum(x[t,l] for l in leaders) == 1)

    # 2. Tour leaders that don't have an American visa cannot give tours in the USA
    model.addConstr(x[0,2] == 0)
    model.addConstr(x[8,2] == 0)

    # 3. Tour leaders must have a three day vacation between tours
    for l in leaders:
        for t in tours:
            for i in range(1, len(tours) - t):
                if start_day_list[t + i] - end_day_list[t] <= a:
                    model.addConstr(x[t+i, l] + x[t, l] <= 1)

    ############## Add Objective ##############
    total_satisfaction = gp.quicksum(x[t,l] * M * cust_sat_matrix_new[t][l] for t in tours for l in leaders)
    total_fee = gp.quicksum(x[t,l] * fee_list[l] * num_customer_list[t] for t in tours for l in leaders)
    objective = total_satisfaction + total_fee
    # objective = w + total_fee
    model.setObjective(objective, gp.GRB.MINIMIZE)
    model.optimize()
    return model, x

def get_solution_df(model, x):
    dict_leaders = {0:"A", 1:"B", 2:"C", 3:"D", 4:"E", 5:"F", 6:"G", 7:"H", 8:"I", 9:"J"}
    # Define the row index (Tour numbers)
    index = range(1, 16)  # 1 to 15

    # Define the column names (excluding "Tour" since it's now the index)
    columns = ["Leader", "Leader Score", "Tour Leader Cost", "Num Customers", "Start", "End"]

    # Create an empty DataFrame with "Tour" as the index
    df = pd.DataFrame(columns=columns, index=index)
    df["Num Customers"] = num_customer_list
    df["Start"] = start_day_list
    df["End"] = end_day_list
    # Rename the index to "Tour"
    df.index.name = "Tour"
    # Start adding values to the df
    for t in tours:
        for l in leaders:
            if x[t,l].x == 1:
                df.loc[t+1,"Leader"] = dict_leaders[l]
                df.loc[t+1,"Leader Score"] = x[t,l].x * cust_sat_matrix[t][l]
                df.loc[t+1,"Tour Leader Cost"] = x[t,l].x * fee_list[l]

    return df

## Test out which M is optimal

In [5]:
############### Test out which M is optimal #############
min_M = 0
min_satisfaction_scores = []
satisfaction_scores = []
for i in range(1, 100000, 100):
    M = i-1 # since we don't want odd numbers like 101, but we want 100, for example
    model, x = optimize_tours(M, 2) # switch the second arguemtn to 0,1,2 or 3 to find the optimal M for each number of vacation days.
    for t in tours:
        for l in leaders:
            if x[t,l].x == 1:
                satisfaction_scores.append(x[t,l].x * cust_sat_matrix[t][l])
    if min(satisfaction_scores) not in min_satisfaction_scores:
        min_satisfaction_scores.append(min(satisfaction_scores))
        if min_satisfaction_scores[-1] == max(min_satisfaction_scores):
            min_M = M
    satisfaction_scores.clear()

print(f"The highest minimum satisfaction score possible to achieve is {max(min_satisfaction_scores)} using M={min_M}")
print(f"The list of minimum satisfaction scores is: {min_satisfaction_scores}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-16
The highest minimum satisfaction score possible to achieve is 70.0 using M=300
The list of minimum satisfaction scores is: [39.0, 42.0, 70.0, 56.0]


## Print the Solution

In [ ]:
############### Print the Solution #############
M = 300
# Solve
model, x = optimize_tours(M)
# get a df to display the solution
df = get_solution_df(model, x)
total_cost_all_customers = sum(x[t,l].x * fee_list[l] * num_customer_list[t] for t in tours for l in leaders)
total_cost_per_customer = sum(x[t,l].x * fee_list[l] for t in tours for l in leaders)
average_score = df["Leader Score"].mean()
min_score = df["Leader Score"].min()
max_sore = df["Leader Score"].max()
print(f"The total cost for all customers is ${total_cost_all_customers}")
print(f"The total cost per customer for all tours is ${total_cost_per_customer}")
print(f"The average customer score is {average_score}%")
print(f"The minimum customer score is {min_score}%")
print(f"The maximum customer score is {max_sore}%")
df

The total cost for all customers is $470400.0
The total cost per customer for all tours is $14400.0
The average customer score is 86.8%
The minimum customer score is 70.0%
The maximum customer score is 99.0%


,Leader,Leader Score,Tour Leader Cost,Num Customers,Start,End
Tour,,,,,,
1,A,90.0,1500.0,30,1,12
2,E,81.0,1100.0,26,1,12
3,C,91.0,1300.0,36,2,13
4,F,92.0,0.0,30,4,14
5,I,97.0,1600.0,36,6,17
6,D,76.0,1100.0,30,7,17
7,G,87.0,0.0,44,8,19
8,B,95.0,1400.0,40,10,21
9,J,80.0,1200.0,26,14,25


### Check correlation between the expected number of customers and the tour leader cost

In [9]:
df_for_corr = df.iloc[:, 2:4].applymap(lambda val: val.x if hasattr(val, "x") else val)
df_for_corr.corr(numeric_only=True)
### As can be seen, tour leader cost is negatively correlated with the number of customers. This means that the more customers there are,
### the cheaper the tourguide will cost to minimize the total cost that customers pay

,Tour Leader Cost,Num Customers
Tour Leader Cost,1.000000,-0.412307
Num Customers,-0.412307,1.000000


# Sensitivity Analysis

Change the number of days that a tour leader has between tours

In [ ]:
# The number of days between tours can only be 0, 1, 2, or 3 for model feasibility because otherwise there will not be enough tour leaders to lead all 15 tours.
M = min_M
min_satisfaction_scores_sens = []   # this list contains the minimum satisfaction scores for each value of a
average_satisfaction_scores_sens = []   # this list contains the average satisfaction scores for each value of a
total_fees_sens = []    # this list contains the total feesacross all customers

# comment out the following for loop if you just want to use M = 300
for a in range(4):
    if a == 0:
        M = 700
    elif a == 1:
        M == 700
    elif a == 2:
        M = 300
    else:
        M = 500

    model, x = optimize_tours(M, a)
    df = get_solution_df(model, x)
    min_satisfaction_scores_sens.append(df["Leader Score"].min())
    average_satisfaction_scores_sens.append(df["Leader Score"].mean())
    total_fee_a = sum(x[t,l].x * fee_list[l] * num_customer_list[t] for t in tours for l in leaders)
    total_fees_sens.append(total_fee_a)

for a in range(4):
    print(f"Days={a}: min score={min_satisfaction_scores_sens[a]}, average score={average_satisfaction_scores_sens[a]:.1f}, total fee=${total_fees_sens[a]:.0f}")

Days=0: min score=80.0, average score=89.8, total fee=$477000
Days=1: min score=80.0, average score=89.8, total fee=$477000
Days=2: min score=70.0, average score=86.8, total fee=$470400
Days=3: min score=39.0, average score=84.7, total fee=$472400
